In [158]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from sklearn.model_selection import train_test_split, StratifiedKFold, GridSearchCV
from sklearn.metrics import precision_score, recall_score, auc, accuracy_score, roc_auc_score,f1_score,log_loss,\
classification_report, roc_curve
from sklearn.linear_model import LogisticRegression

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [159]:
DATA_DIR = '/kaggle/input/sf-scoring/'
df_train = pd.read_csv(DATA_DIR +'/train.csv')
df_test = pd.read_csv(DATA_DIR +'/test.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [160]:
sample_submission.shape

In [161]:
df_test.shape

In [162]:
df_train.info()

In [163]:
df_train.head(5)

In [164]:
df_test.info()

In [165]:
sample_submission.head(5)

In [166]:
sample_submission.info()

Описание данных:
client_id - идентификатор клиента
app_date - дата подачи заявки
education - уровень образования cat_cols
sex - пол заемщика bin
age - возраст заемщика num_cols
car - флаг наличия автомобиля bin
car_type - флаг автомобиля иномарки bin
decline_app_cnt - количество отказанных прошлых заявок cat_cols
good_work - флаг наличия “хорошей” работы bin
score_bki - скоринговый балл по данным из БКИ num_cols
bki_request_cnt - количество запросов в БКИ num_cols
region_rating - рейтинг региона cat_cols
home_address - категоризатор домашнего адреса cat_cols
work_address - категоризатор рабочего адреса cat_cols
income - доход заемщика num_cols
sna - связь заемщика с клиентами банка cat_cols
first_time - давность наличия информации о заемщике cat_cols
foreign_passport - наличие загранпаспорта bin
default - флаг дефолта по кредиту

In [167]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0  # помечаем где у нас тест
df_test['default'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [168]:
# Посмотрим на пропуски
data.isnull().sum()

In [169]:
#Посмотрим сбалансированность целевого признака
data['default'].value_counts(ascending=True).plot(kind='barh')

In [170]:
#смотрим на распределение признака education
data.education.hist()

In [171]:
data.education.value_counts()

In [172]:
# Заполним education самым распространенным значением.
data.fillna(data.education.value_counts().index[0], inplace=True)

In [173]:
data.nunique(dropna=False)

In [174]:
num_cols = ['age', 'score_bki', 'decline_app_cnt', 'bki_request_cnt', 'income']
cat_cols = ['education', 'first_time', 'sna', 'work_address', 'home_address', 'region_rating']
bin_cols = ['sex', 'car', 'car_type', 'good_work', 'foreign_passport']

# Бинарные переменные

In [175]:
# Для бинарных признаков мы будем использовать LabelEncoder
label_encoder = LabelEncoder()
for column in bin_cols:
    data[column] = label_encoder.fit_transform(data[column])
    
# убедимся в преобразовании    
data[bin_cols].head()

# Категориальные переменные

education он единственный не числовой, он содержит 5 уровней образования. region_rating - имеет 7 уникальных значений. Для них выполним ранжирование.

*"SCH" - школьное "UGR" - бакалавр, "GRD - профессиональное", "PGR" - аспирантура, "ACD" - академическое

In [176]:
# список уникальных значений по возростанию
grade = ['SCH', 'UGR', 'GRD', 'PGR', 'ACD'] 
# трансформация
data['education'] = OrdinalEncoder(categories=[grade]).fit_transform(data[['education']]).astype(int)

# отсортированный список уникальных значений по возростанию   
rating = sorted(data.region_rating.unique().tolist()) 
# трансформируем
data["region_rating"] = OrdinalEncoder(categories=[rating]).fit_transform(data[["region_rating"]]).astype(int)

In [177]:
# убедимся в преобразовании    
display(data['education'].value_counts())
data["region_rating"].value_counts()

In [178]:
data.head()

In [179]:
# теперь посмотрим распределение категориальных и бинарных признаков в зависимости от целевого признака
# графики показывают относительное распределение признаков по каждой группе клиентов: "хороших" и "плохих"
#df = data[data['sample']]
#for col in bin_cols + cat_cols:
  #  plt.figure(figsize=(8, 4))
  #  prop_df = df[col].groupby(df["default"]).value_counts(
   #     normalize=True).rename("percent").reset_index()
  #  prop_df["percent"] *= 100
  #  sns.barplot(x=col, y="percent", hue="default", data=prop_df)
 #   plt.title(f"Распределение признака {col}")

Выводы по категориальным признакам:

sex: женщин в выборке немного больше, чем мужчин; женщины более часто отдают кредит, чем мужчины, но разница незначительна

car: клиентов с машиной примерно в 2 раза меньше, чем без машины. Те, у кого есть машина, более кредитоспособны

car_type: клиентов, у которой машина иномарка, еще меньше, и они также более кредитоспособны.

good_work: клиентов с хорошей работой значительно меньше, чем остальных и они более кредитоспособны

foreign_passport: клиентов с загранпаспортом также значительно меньше остальных и они тоже более кредитоспособны education: чем выше уровень образования у клиента, тем выше вероятность вернуть кредит

region_rating: ординальный признак, принимающий значение от 0 до 4

home_address: домашние адреса распределены по 3 категориям. Клиенты, проживающие в 1 категории адреса более кредитоспособны, чем во второй и в третьей

work_address: рабочие адреса также распределены по 3 категориям. Клиенты, работающие в 1 и 2 категориях адреса более кредитоспособны, чем работающие в 3 категории

sna (связь заемщика с клиентами банка): чем выше показатель, тем выше риск невозврата кредита

first_time (давность наличия информациии о заемщике): чем выше показатель, тем более благонадежен клиент

# Числовые признаки

In [180]:
#функция для построения графиков

def get_boxplot(column):
    fig, ax = plt.subplots(figsize=(14, 4))
    sns.boxplot(x=column,
                data=data,
                ax=ax)
    plt.xticks(rotation=0)
    ax.set_title('Boxplot for ' + column)
    plt.show()

In [181]:
#построим графики чтобы посмотреть в каких столбцах у нас есть выбросы.
for col in num_cols:
    get_boxplot(col)

In [182]:
for i in num_cols:
    plt.figure()
    sns.distplot(data[i], kde = False, rug=False)
    plt.title(i)
    plt.show()

Числовые переменные (кроме score_bki, crime) имеют выбросы и/либо длинный правый хвост. Выполним логарифмирование для того чтобы распеределение стало более нормальным

In [183]:
log_list = ['age', 'decline_app_cnt', 'bki_request_cnt', 'income']
for col in log_list:
  data[col] = np.log(data[col] + 1)

In [184]:
for i in num_cols:
    plt.figure()
    sns.distplot(data[i], kde = False, rug=False)
    plt.title(i)
    plt.show()

# Обработка дат

In [185]:
# дата - сразу сконветируем формат даты к удобному для работы
#data.app_date = pd.to_datetime(data.app_date)
# Выведем количество дней и приведем к числовому призанку
#data['app_date'] = (data.app_date - data.app_date.min()).dt.days.astype(int)
#num_cols.append('app_date')

In [186]:
num_cols

# Корреляция

In [187]:
# посмотрим на корреляцию признаков между собой
plt.figure(figsize=(12, 10))
sns.heatmap(data[cat_cols+bin_cols+num_cols].corr().abs(), vmin=0,
            vmax=1, annot=True, fmt=".2f", cmap="YlGnBu")

# 3. Feature engineering

Преобразуем в get_dummies категориальные признаки

In [188]:
# объеденим адреса, т.к. они сильно коррелируют
data['address'] = (data.work_address*data.home_address)**2
cat_cols.append('address')
cat_cols.remove('home_address')
cat_cols.remove('work_address')
data.drop(['home_address', 'work_address'], axis=1, inplace=True)

# объеденим car и car_type и переведем в категориальный признак
data['car_all'] = data['car'] + data['car_type']
data.drop(['car', 'car_type'], axis=1, inplace=True)
cat_cols.append('car_all')
bin_cols.remove('car')
bin_cols.remove('car_type')

In [189]:
# Новый признак - месяц
data['app_date'] = pd.to_datetime(data['app_date'], format='%d%b%Y')
data['month'] = data['app_date'].dt.month.astype(object)

data.drop(['app_date'],  axis=1, inplace=True)

In [190]:
# Средний доход с учетом рейтинга региона

mean_inc_reg = data.groupby('region_rating')['income'].median().to_dict()
data['mean_income_region'] = data['region_rating'].map(mean_inc_reg)

In [191]:
# Средний доход с учетом возраста

mean_inc_age = data.groupby('age')['income'].median().to_dict()
data['mean_income_age'] = data['age'].map(mean_inc_age)

In [192]:
# Средний score bki с учетом возраста

mean_bki_age = data.groupby('age')['score_bki'].median().to_dict()
data['mean_bki_age'] = data['age'].map(mean_bki_age)

In [193]:
# посмотрим на количество столбцов
data.shape

In [194]:
# посмотрим на корреляцию числовых признаков между собой
plt.figure(figsize=(8, 5))
sns.heatmap(data[num_cols].corr().abs(), vmin=0,
            vmax=1, annot=True, fmt=".2f", cmap="YlGnBu")

In [195]:
# посмотрим на корреляцию бинарных признаков между собой
plt.figure(figsize=(8, 5))
sns.heatmap(data[bin_cols].corr().abs(), vmin=0,
            vmax=1, annot=True, fmt=".2f", cmap="YlGnBu")

In [196]:
data.drop(['client_id'], axis = 1, inplace=True)

In [197]:
data = pd.get_dummies(data, columns=['education'], dummy_na=True)

In [198]:
data.info()

In [199]:
data

In [200]:
# Теперь выделим тестовую часть
train_data = data.query('sample == 1').drop(['sample'], axis=1)
test_data = data.query('sample == 0').drop(['sample', 'default'], axis=1)

y = train_data['default'].values  # наш таргет
X = train_data.drop(['default'], axis=1)

In [201]:
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [202]:
# проверяем
X_train.shape, X_test.shape, y_train.shape, y_test.shape

In [203]:
logreg = LogisticRegression(solver='liblinear', max_iter=1000)
logreg.fit(X_train, y_train)
y_pred = logreg.predict(X_test)
y_score = logreg.predict_proba(X_test)[:,1]

In [204]:
# если качество нас устраивает, обучаем финальную модель на всех обучающи данных
logreg_final = LogisticRegression(solver='liblinear', max_iter=1000)
logreg_final.fit(X, y)

In [205]:
predict_submission = logreg_final.predict(test_data)

In [206]:
sample_submission['default'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

In [207]:
sample_submission.describe()

In [208]:
!kaggle competitions submit -c sf-scoring -f ssubmission.csv -m "Message"
# !kaggle competitions submit your-competition-name -f submission.csv -m 'My submission message'